# Notebook projet ia

In [17]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

gd = pd.read_csv("./data/general_data.csv")
esd = pd.read_csv("./data/employee_survey_data.csv")
in_time = pd.read_csv("./data/in_time.csv")
out_time = pd.read_csv("./data/out_time.csv")

# Nétoyage des données non ethique

In [18]:
data = gd.copy()
data.join(esd, rsuffix="_esd")
data.drop(columns=["EmployeeCount", "EmployeeID", "Gender", "MaritalStatus", "Over18", "StandardHours"], axis=1, inplace=True)

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = data.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='median')),
      ('scaler', StandardScaler())])

ohe = OneHotEncoder(handle_unknown='ignore')

categorical_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
      ('onehot', ohe)])

preprocessor = ColumnTransformer(
      transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
)

# Create the pipeline for preprossessing and classifying
data_prepared = preprocessor.fit_transform(data)


In [81]:
data2 = data.loc[:, categorical_features]
data3 = data.loc[:, numeric_features]
data2_prepared = pd.get_dummies(data2)

data4 = data2_prepared.join(data3)
cor_att = data4.corr()[:1]

cor_att.where(abs(cor_att) > 0.05, inplace=True)
cor_att.dropna(inplace=True, axis=1)
cor_att.head()


,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,Department_Human Resources,EducationField_Human Resources,Age,TotalWorkingYears,YearsAtCompany,YearsWithCurrManager
Attrition_No,1.0,-1.0,0.074457,-0.115143,-0.080768,-0.091573,0.159205,0.170338,0.134392,0.156199
